# Airflow Folder — Technical & Implementation Documentation  
**Path:** `final_capstone/airflow`  
**Purpose:** Automates and orchestrates the full MLOps workflow for sales conversion prediction using Apache Airflow.

---

## 1. **Architecture & Flow**

### **Pipeline Overview**
The Airflow folder is designed to run a **production-grade, automated MLOps pipeline**. It manages:
- Data extraction from databases
- Preprocessing and feature engineering
- Exploratory Data Analysis (EDA)
- Model training, selection, and registration
- Data drift detection and retraining
- Dashboard updates and reporting

### **Typical DAG Flow**
1. **Extract Data**  
   - Source: PostgreSQL/pgAdmin  
   - Operator: PythonOperator or PostgresOperator  
   - Output: Raw CSV in `data/`

2. **Preprocess Data**  
   - Cleans, encodes, transforms  
   - Saves processed data, pipeline, and metadata

3. **EDA**  
   - Generates plots, stats, and saves reports

4. **Model Training & Selection**  
   - Trains multiple models, tunes hyperparameters  
   - Selects best model, saves artifacts

5. **Drift Detection**  
   - Compares new data to training data  
   - Logs drift status, triggers retraining if needed

6. **Dashboard Update**  
   - Refreshes Streamlit dashboard with latest results

7. **Logging & Monitoring**  
   - Logs all steps for audit and debugging

---

## 2. **Key Files & Their Roles**

### **Configuration**
- `.env`: Stores secrets (DB credentials, paths)
- `docker-compose.yml`: Multi-container setup for Airflow, DB, MLflow, etc.
- `Dockerfile`: Customizes Airflow image (Python, pip packages)

### **DAGs (`dags/` folder)**
Each DAG is a Python file defining a workflow:

- **`extract_pgadmin.py`**  
  Extracts data from PostgreSQL using SQL queries.  
  *Why:* Ensures fresh, consistent data for ML.

- **`preprocess.py`**  
  Cleans and transforms raw data using a reproducible pipeline.  
  *Why:* Guarantees consistency between training and inference.

- **`eda.py`**  
  Runs automated EDA, saves visualizations and metrics.  
  *Why:* Provides insights for feature engineering and business understanding.

- **`mlops_full_processs.py` / `main_full_processs.py`**  
  Orchestrates the complete ML workflow: extraction, preprocessing, training, evaluation, and artifact saving.  
  *Why:* Automates end-to-end ML lifecycle.

- **`conplete_mlops.py`**  
  (Typo for `complete_mlops.py`) Runs the full MLOps pipeline as a single DAG.

- **`modal.py`**  
  Handles model training, selection, and saving.

- **`data_drift_check.PY`**  
  Monitors for data drift using statistical tests (e.g., KS test, PSI), logs drift status, triggers retraining.

- **`streamlit_dash.py`**  
  Updates Streamlit dashboard with latest results.

### **Data & Artifacts (`data/` folder)**
- Raw and processed datasets (`Lead Scoring.csv`, `check.csv`, `preprocessed_data.csv`)
- Saved pipeline (`preprocessing_pipeline.pkl`)
- Metadata (`processing_metadata.pkl`)
- Model artifact (`best_model_xgboost.pkl`)
- Drift logs (`drift_detected.txt`, `drift_log.txt`, `drift_status.flag`)
- Dashboard logs (`streamlit.log`)

### **Logs (`logs/` folder)**
- Organized by DAG ID, contains execution logs for each pipeline run

---

## 3. **How to Implement**

### **Step 1: Setup Environment**
- Install Docker and Docker Compose
- Clone the repository
- Configure `.env` with DB credentials and paths

### **Step 2: Build & Start Services**


In [ ]:
docker-compose up --build

- Starts Airflow webserver, scheduler, database, MLflow, etc.

### **Step 3: Configure Airflow**
- Access Airflow UI at `http://localhost:8080`
- Set up connections (e.g., PostgreSQL, S3) in Airflow UI

### **Step 4: DAG Implementation Example**


In [ ]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime
import pandas as pd
import pickle

def preprocess_data():
    df = pd.read_csv('/data/Lead Scoring.csv')
    # ...cleaning, encoding, transformation...
    with open('/data/preprocessing_pipeline.pkl', 'wb') as f:
        pickle.dump(preprocessor, f)
    df.to_csv('/data/preprocessed_data.csv', index=False)

with DAG('preprocess_lead_data', start_date=datetime(2023, 1, 1), schedule_interval='@daily') as dag:
    preprocess_task = PythonOperator(
        task_id='preprocess_data',
        python_callable=preprocess_data
    )

- **Reason:** Modularizes preprocessing for reuse and scheduling.

### **Step 5: Monitor & Debug**
- Check Airflow UI for DAG status and logs
- Review output files in `data/` and logs in `logs/`

### **Step 6: Extend Pipeline**
- Add new DAGs for model monitoring, explainability, or retraining
- Integrate with cloud storage (S3, Azure Blob) for data and artifacts
- Connect to CI/CD for automated deployment

---

## 4. **Technical Importance**

- **Automation:**  
  Airflow schedules and monitors all steps, reducing manual work.
- **Reproducibility:**  
  Pipelines, models, and metadata are versioned and saved.
- **Scalability:**  
  Docker and Airflow support distributed, parallel execution.
- **Auditability:**  
  Logs and artifacts enable traceability and compliance.
- **Reliability:**  
  Drift detection and retraining maintain model performance.
- **Business Alignment:**  
  Dashboard and reporting keep stakeholders informed.

---

## 5. **Extensibility**

- Add new DAGs for additional ML tasks (e.g., model monitoring, explainability)
- Integrate with cloud storage for input/output
- Enhance drift detection with advanced statistical or ML methods
- Automate retraining and reporting based on business needs

---

## 6. **Summary Table**

| Component         | Purpose                                  | Implementation/Reason                |
|-------------------|------------------------------------------|--------------------------------------|
| `.env`            | Store secrets/config                     | Secure, flexible configuration       |
| `docker-compose.yml` | Multi-container orchestration         | Scalable, reproducible environment   |
| `dags/`           | Pipeline steps as DAGs                   | Modular, maintainable workflow       |
| `data/`           | Data and model artifacts                 | Versioned, auditable outputs         |
| `logs/`           | Execution logs                           | Debugging, compliance                |
| Drift Detection   | Monitor data changes                     | Maintain model reliability           |
| Dashboard         | Business insights                        | Stakeholder communication            |

---

## 7. **Conclusion**

The Airflow folder provides a **complete, automated, and production-ready MLOps orchestration** for sales conversion prediction. It ensures data integrity, model reliability, and business alignment through modular DAGs, robust logging, and automated drift monitoring.

---

